In [181]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import tqdm
import numpy as np
import pickle
import random

from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

from random import randint



In [23]:
os.getcwd()

'/raid/home/arend036/msc_pim/Thesis-Scripts'

In [35]:
WORD_EMBEDDING_PATH = '/mnt/guanabana/raid/data/datasets/GloVe/pretrained' 
GLOVE_PATH = WORD_EMBEDDING_PATH + '/glove.42B.300d.txt'
CAVS_PATH = '../data/filtered_broden_cavs.pickle'

In [36]:
with open(CAVS_PATH, 'rb') as handle:
        cavs_broden = pickle.load(handle)

In [37]:
# takes about 4min to run on guanabana

embedding_dict = {} # 1.9M words

with open(GLOVE_PATH, 'r', encoding="utf-8") as f:
    for line in tqdm.notebook.tqdm(f):
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], 'float32')
        embedding_dict[word] = vector

In [47]:
concepts = list(cavs_broden.keys())
embedding_concepts = []

for c in concepts:
    if c in embedding_dict.keys():
        embedding_concepts.append(c)

In [48]:
len(embedding_concepts)

321

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

For all the concepts which have a concept activation vector and have a word embedding, the CAVs are concatenated into a matrix. Also the matching word embeddings are concatenated into a matrix. <br>
These are then split in 75% training and 25% test data and converted into a TensorDataset

In [112]:
x_cavs = torch.from_numpy(cavs_broden[embedding_concepts[0]]['cav']).float()
y_word_vec = torch.from_numpy(embedding_dict[embedding_concepts[0]]).unsqueeze(0).float()

for i in range(1, len(embedding_concepts)):
    cav = torch.from_numpy(cavs_broden[embedding_concepts[i]]['cav']).float()
    x_cavs = torch.cat((x_cavs, cav), 0)
    
    word_vec = torch.from_numpy(embedding_dict[embedding_concepts[i]]).unsqueeze(0).float()
    y_word_vec = torch.cat((y_word_vec, word_vec), 0)


In [177]:
# randomly sample 25% indices of the matrices 
random.seed(42)
random_idxs = random.sample(range(len(embedding_concepts)), 80)


X_train = np.delete(x_cavs, random_idxs, axis=0)
X_test = x_cavs[random_idxs,:]

y_train = np.delete(y_word_vec, random_idxs, axis=0)
y_test = y_word_vec[random_idxs,:]

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size = 10)
test_loader = DataLoader(test_dataset, batch_size = 10)

Create a simple neural network which is a linear function between the concept activation vectors and the word embeddings

In [179]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(2048, 300)
        
    def forward(self, x):
        x = self.fc1(x)
        return x

net = Net()        

Define the optimizer and the loss function

In [189]:
optimizer = optim.Adam(net.parameters(), lr=0.0001)
criterion = nn.MSELoss()

Train the neural network

In [190]:
for epoch in range(10):
    
    running_loss = 0.0
    for data in train_loader:
        X, y = data
        
        optimizer.zero_grad()
        
        outputs = net(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
    
    print(loss)
        

tensor(4.7040, grad_fn=<MseLossBackward>)
tensor(1.6489, grad_fn=<MseLossBackward>)
tensor(0.7474, grad_fn=<MseLossBackward>)
tensor(0.4251, grad_fn=<MseLossBackward>)
tensor(0.2715, grad_fn=<MseLossBackward>)
tensor(0.1880, grad_fn=<MseLossBackward>)
tensor(0.1369, grad_fn=<MseLossBackward>)
tensor(0.1023, grad_fn=<MseLossBackward>)
tensor(0.0777, grad_fn=<MseLossBackward>)
tensor(0.0595, grad_fn=<MseLossBackward>)


In [201]:
MODEL_PATH = '../models/cav_to_word_net.pth'
torch.save(net.state_dict(), MODEL_PATH)

In [227]:
weights = list(net.parameters())[0]

bias = net.fc1.bias

In [202]:
embedding_concepts.index('mountain')

23

In [239]:
mountain_emb = embedding_dict['mountain'].reshape(300,1)

In [243]:
t = (torch.from_numpy(mountain_emb) - bias.unsqueeze(1)) / weights

In [248]:
bias.unsqueeze(1).shape

torch.Size([300, 1])